In [1]:
import pandas as pd
import re

In [2]:
old_surveys_final = pd.read_csv('../../data/old_pgls_final.csv')

In [3]:
old_surveys_final.rename(columns={'nps': 'NPS'}, inplace=True)

old_surveys_final.to_csv('../../data/old_pgls_final.csv', index=False)